In [1]:
import cv2
import numpy as np
import csv
import time
import matplotlib.pyplot as plt

In [ ]:
pip uninstall opencv-python-headless

In [20]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
def display_image(frame):
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [3]:
def get_quadrant(x, y, width, height):
    if x < width / 2 and y < height / 2:
        return 4  
    elif x >= width / 2 and y < height / 2:
        return 3  
    elif x < width / 2 and y >= height / 2:
        return 1 
    else:
        return 2 

In [4]:
def detect_color(hsv_frame, x, y, w, h):
    roi = hsv_frame[y:y+h, x:x+w]
    avg_color_per_row = np.average(roi, axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    avg_hue = avg_color[0]

    if 0 <= avg_hue <= 10 or 160 <= avg_hue <= 180:
        return "Red"
    elif 35 <= avg_hue <= 85:
        return "Green"
    elif 100 <= avg_hue <= 130:
        return "Blue"
    elif 20 <= avg_hue <= 30:
        return "Yellow"
    else:
        return "Unknown"

In [5]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [6]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [7]:
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [8]:
ball_class_id = classes.index("sports ball")

In [9]:
# Dictionary to track last known quadrant of each ball
ball_quadrants = {}

In [10]:
events = []

In [11]:
trackers = []

In [12]:
#video capture
cap = cv2.VideoCapture("AI Assignment video.mp4")  # Or use 0 for webcam

In [14]:
with open("events_log.txt", mode="w") as f:
    f.write("Time, Quadrant Number, Ball Colour, Type\n")

    # Loop frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        height, width, channels = frame.shape
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)
        timestamp = time.time()  # Current timestamp

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == ball_class_id:  # Only consider balls
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                color = detect_color(hsv_frame, x, y, w, h)
                label = f"Ball - {color}"
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f"{label}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                center_x, center_y = x + w // 2, y + h // 2
                quadrant = get_quadrant(center_x, center_y, width, height)

                # Tracking entry and exit events
                if (x, y, w, h) in ball_quadrants:
                    last_quadrant = ball_quadrants[(x, y, w, h)]
                    if last_quadrant != quadrant:
                        # exit event for the last quadrant
                        f.write(f"{timestamp}, {last_quadrant}, {color}, Exit\n")
                        # entry event for the new quadrant
                        f.write(f"{timestamp}, {quadrant}, {color}, Entry\n")
                        print(f"Ball {color} exited quadrant {last_quadrant} and entered quadrant {quadrant} at {timestamp:.2f}s")
                else:
                    # initial entry event
                    f.write(f"{timestamp}, {quadrant}, {color}, Entry\n")
                    print(f"Ball {color} entered quadrant {quadrant} at {timestamp:.2f}s")

                # last known quadrant of the ball
                ball_quadrants[(x, y, w, h)] = quadrant

        cv2.imshow("Frame", frame)
        
        key = cv2.waitKey(1)
        if key == 27: 
            break


Ball Yellow entered quadrant 3 at 1720446617.87s
Ball Yellow entered quadrant 3 at 1720446619.65s
Ball Yellow entered quadrant 3 at 1720446620.23s
Ball Yellow entered quadrant 3 at 1720446630.68s
Ball Yellow entered quadrant 3 at 1720446633.23s
Ball Green entered quadrant 3 at 1720446633.23s
Ball Green entered quadrant 3 at 1720446633.98s
Ball Yellow entered quadrant 3 at 1720446635.18s
Ball Yellow entered quadrant 3 at 1720446635.83s
Ball Yellow entered quadrant 3 at 1720446637.85s
Ball Yellow entered quadrant 3 at 1720446639.12s
Ball Yellow entered quadrant 3 at 1720446641.10s
Ball Yellow entered quadrant 3 at 1720446641.72s
Ball Yellow entered quadrant 3 at 1720446642.42s
Ball Yellow entered quadrant 3 at 1720446643.07s
Ball Yellow entered quadrant 3 at 1720446644.50s
Ball Yellow entered quadrant 3 at 1720446647.79s
Ball Yellow entered quadrant 3 at 1720446648.50s
Ball Yellow entered quadrant 3 at 1720446649.82s
Ball Yellow entered quadrant 3 at 1720446652.97s
Ball Yellow entered qu

In [15]:
cap.release()
cv2.destroyAllWindows()